In [ ]:
import sys
!{sys.executable} -m pip install storysniffer
!{sys.executable} -m pip install newspaper3k
!{sys.executable} -m pip install -U dill
!{sys.executable} -m pip install scikit-learn==1.0.2

In [ ]:
import requests
from urllib.parse import urlparse, urljoin
import sys
from bs4 import BeautifulSoup
import re
from datetime import datetime
import json
import urllib.parse
import hashlib
from storysniffer import StorySniffer
import ast
from newspaper import Article
import nltk
import pandas as pd

def is_valid(url):
    """
    Checks whether `url` is a valid URL.
    """
    parsed = urlparse(url)
    return bool(parsed.netloc) and bool(parsed.scheme)


def get_news_urls(url):
    
    # all URLs of `url` 
    domain_name = urlparse(url).netloc
    urls = set()
    response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko; Google Page Speed Insights) Chrome/41.0.2272.118 Safari/537.36'})
    soup = BeautifulSoup(response.content, "html.parser")
    for a_tag in soup.findAll("a"):
        href = a_tag.attrs.get("href")
        if href == "" or href is None:
            # href empty tag
            continue
        # join the URL if it's relative (not absolute link)
        href = urljoin(url, href)
        parsed_href = urlparse(href)
        # remove URL GET parameters, URL fragments, etc.
        href = parsed_href.scheme + "://" + parsed_href.netloc + parsed_href.path
        if not is_valid(href):
            # not a valid URL
            continue
        if href in internal_urls:
            # already in the set
            continue
        if domain_name not in href:
            # external link
            if href not in external_urls:
                external_urls.add(href)
            continue
        urls.add(href)
        internal_urls.add(href)      
        
def crawl(news_url):
    print('get link ==> ', news_url)
    resp = requests.get(news_url, headers={'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko; Google Page Speed Insights) Chrome/41.0.2272.118 Safari/537.36'})
    return resp.text

def get_host(url: str):
    token = url.split('://')[1]
    token = token.split('.')
    if token[0] == 'www':
        return token[1]
    return token[0]

In [ ]:
internal_urls = set()
external_urls = set()
    
url = 'https://www.studlife.com/news/'

urls = get_news_urls(url)

valid_urls = []
    
sniffer = StorySniffer()

for u in internal_urls:
    try:
        if ".pdf" in u: 
            continue
        if "/event" in u: 
            continue
        if "/places" in u: 
            continue
        if "/classifieds" in u:
            continue
        if "/special_sections" in u:
            continue
        if "/form" in u:
            continue
        if "/edition" in u:
            continue
        if "/archive" in u:
            continue
        if "/feed" in u:
            continue
        if "gallery" in u:
            continue
        if "/ads" in u:
            continue
        if "/promo" in u:
            continue
        if "/video" in u:
            continue
        if "/slideshow" in u:
            continue
        if "/members-only" in u:
            continue
#Video only series from the STL Post-Dispatch
        if "/the-bottom-line" in u:
            continue
#Removing non-article pages from the Suburban Journals of Greater St. Louis
        if (urllib.parse.urlsplit(u).hostname == 'www.stltoday.com') and (not '/article' in u):
            continue
#Removing the landing pages from the St. Louis Review
        if (urllib.parse.urlsplit(u).hostname == 'www.archstl.org') and (not u[-4:].isnumeric()):
            continue
#Removing landing pages from the Sullivan Independent News
        if (urllib.parse.urlsplit(u).hostname == 'www.mysullivannews.com') and ("bourbon" in u):
            continue
#Removing landing pages from The Fulton Sun
        if (urllib.parse.urlsplit(u).hostname == 'www.fultonsun.com') and (('/fulton-news/' in u) or (not '-' in u)):
            continue
#Removing image only pages from The Joplin Globe
        if (urllib.parse.urlsplit(u).hostname == 'www.joplinglobe.com') and ('/image' in u):
            continue
#Removing calendar pages from The Marshall Democrat-News
        if (urllib.parse.urlsplit(u).hostname == 'www.marshallnews.com') and ('-calendar' in u):
            continue
#Removing landing pages from the Maryville Forum
        if (urllib.parse.urlsplit(u).hostname == 'www.maryvilleforum.com') and (('/multimedia/photos' in u) or (not '.html' in u)):
            continue
#Removing landing pages from The Pitch
        if (urllib.parse.urlsplit(u).hostname == 'www.thepitchkc.com') and (('/city-rag-kansas-city-news' in u) or ('/dessert-week-2022' in u)):
            continue
#Removing advertising pages from The Standard
        if (urllib.parse.urlsplit(u).hostname == 'www.the-standard.org') and ('/advertise' in u):
            continue
#Removing landing pages from the Webster Kirkwood News
        if (urllib.parse.urlsplit(u).hostname == 'websterkirkwoodtimes.com') and (not '/article' in u):
            continue
#Removing landing pages from the West End Word
        if (urllib.parse.urlsplit(u).hostname == 'www.timesnewspapers.com') and (not '/article' in u):
            continue
#Removing landing pages from KRZK
        if (urllib.parse.urlsplit(u).hostname == 'www.legends1063.fm') and ('/advertise-with-us' in u):
            continue
#Removing landing pages from KCUR and KSMU
        if (urllib.parse.urlsplit(u).hostname in ['www.kcur.org', 'www.ksmu.org', 'www.krcu.org']) and (re.findall(r'\d{4}-\d{2}-\d{2}', u) == []):
            continue
#Removing landing pages from News Press Now
        if (urllib.parse.urlsplit(u).hostname == 'www.newspressnow.com') and (not '.html' in u):
            continue
#Removing landing pages from KTTS
        if (urllib.parse.urlsplit(u).hostname == 'www.ktts.com') and (not '-' in u):
            continue
#Removing landing pages from KPRS
        if (urllib.parse.urlsplit(u).hostname == 'www.kprs.com') and (re.findall(r'\d{4}', u) == []):
            continue
#Removing landing pages from My MO Info
        if (urllib.parse.urlsplit(u).hostname == 'www.mymoinfo.com') and ('/trading-post' in u):
            continue
#Removing landing pages from KBIA
        if (urllib.parse.urlsplit(u).hostname == 'www.kbia.org') and (('/show' in u) or ('/podcast' in u) or ('paul-pepper' in u) or (re.findall(r'\d{4}-\d{2}-\d{2}', u) == [])):
            continue
#Removing podcast landing pages from 97.1FM and 98.1FM
        if (urllib.parse.urlsplit(u).hostname == 'www.audacy.com') and (('/podcast' in u) or ('/author' in u)):
            continue
#Removing non-news from The Beat
        if (urllib.parse.urlsplit(u).hostname == 'www.digitalburg.com') and (not '/central-news' in u):
            continue
#Removing non-news from Farmer Publishing
        if (urllib.parse.urlsplit(u).hostname == 'farmerpublishing.com') and ('e-mail-newsletter' in u):
            continue
#Removing landing pages from Benton County Enterprise
        if (urllib.parse.urlsplit(u).hostname in ['www.bentoncountyenterprise.com']) and (not re.findall(r'\d{4}', u) == []):
            continue
        if (urllib.parse.urlsplit(u).hostname in ['lewispnj.com', 'maconhomepress.com', 'www.theijnews.com']) and ('/articles' in u):
            valid_urls.append(u)
        if (urllib.parse.urlsplit(u).hostname == 'www.thecash-book.com') and (len(u)<55):
            continue
        if (urllib.parse.urlsplit(u).hostname in ['www.hermannadvertisercourier.com', 'www.lakegazette.net']) and (re.findall(r'\d', u) == []):
            continue
        if (urllib.parse.urlsplit(u).hostname in ['bocojo.com']) and (('/news' in u) or ('/obituaries' in u) or ('/sports' in u) or ('/opinion' in u)) and (not '/category' in u):
            valid_urls.append(u)    
        if (urllib.parse.urlsplit(u).hostname in ['dosmundos.com', 'newstalkkzrg.com', 'www.949kcmo.com']) and (re.findall(r'\d{4}\/', u) == []):
            continue 
        if (urllib.parse.urlsplit(u).hostname in ['www.redlatinastl.com']) and ('/noticia' in u):
            valid_urls.append(u)   
        if (urllib.parse.urlsplit(u).hostname in ['emissourian.com']) and (('/marketplace' in u) or (re.findall(r'\d', u) == [])):
            continue 
        if (urllib.parse.urlsplit(u).hostname in ['molawyersmedia.com', 'store.molawyersmedia.com']) and (('/submit' in u) or ('store.' in u) or (re.findall(r'\d{4}\/', u) == [])):
            continue 
            
        if sniffer.guess(u) == True and (not u in valid_urls):
            valid_urls.append(u)

    except BaseException as e:
        pass
    except SSLError:
        pass
    except MaxRetryError:
        pass
    except SSLCertVerificationError:
        pass

In [ ]:
import sklearn

print('The scikit-learn version is {}.'.format(sklearn.__version__))

In [ ]:
internal_urls

In [ ]:
valid_urls

In [ ]:
news_data = []
# print(valid_urls)
try:
    for u in valid_urls:
        hash = hashlib.sha256(u.encode())
        news_data.append({
            'id': hash.hexdigest(),
            'url': u,
            'crawl_date': str(datetime.now()),
            'text': crawl(u),
            'host': urllib.parse.urlsplit(u).hostname
            })
except BaseException as e:
    pass

In [ ]:
news_data

In [ ]:
raw_articles = news_data

len(raw_articles)

In [ ]:
article = raw_articles[0]


In [ ]:
def given_tags(item,data):
    
    if data['hostname'] in ['www.bethanyclipper.com']:
        tags = []
        soup = BeautifulSoup(article['text'],'html.parser')
        key = soup.find("span", attrs={"class":"tag-links"})
        try:
            for x in key.find_all('a'):
                tags.append(x.get_text())
            return tags
        except AttributeError:
            return ''
    
    if data['hostname'] in ['www.excelsiorspringsstandard.com']:
        tags = []
        soup = BeautifulSoup(article['text'],'html.parser')
        key = soup.find("ul", attrs={"class":"links field__items"})
        tag = key.find('a').get_text()
        tags.append(tag) 
        return [tags[0]]
    
    if data['hostname'] in ['www.dixonpilot.com', 'www.edinasentinel.com']:
        tags = []
        soup = BeautifulSoup(article['text'],'html.parser')
        key = soup.find("div", attrs={"class":"fl-post-cats-tags"})
        tag = key.find('a').get_text()
        tags.append(tag) 
        return [tags[0]]
    
    if data['hostname'] in ['missouri.statenews.net', 'www.kansascity.com', 'www.kxcv.org']:
        tags = []
        soup = BeautifulSoup(article['text'],'html.parser')
        key = soup.find(attrs={"name":"keywords"})
        keyword = (key.get("content", None))
        tags.append(keyword) 
        return tags
    
    if data['hostname'] == 'www.949kcmo.com':
        tags = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            key = soup.find_all("meta", attrs={"property":"article:section"})
            for x in key:
                keyword = (x.get("content", None))
                tags.append(keyword) 
            return tags
           
        except AttributeError:
            return ''
        
        except TypeError:
            return ''

    if data['hostname'] == 'unewsonline.com':
        tags = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            key = soup.find("meta", attrs={"property":"article:section"})
            keyword = (key.get("content", None))
            tags.append(keyword) 
            return tags[0]
           
        except AttributeError:
            return ''
        
        except TypeError:
            return ''

    if data['hostname'] in ['molawyersmedia.com']:
        tags = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            key = soup.find("meta", attrs={"name":"keywords"})
            keyword = (key.get("content", None))
            keyword2 = keyword.split(",")
            tags=keyword2 
            return tags
           
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
        
    if data['hostname'] in ['www.hermannadvertisercourier.com', 'www.lakegazette.net', 'emissourian.com']:
        tags = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            key = soup.find("meta", attrs={"name":"news_keywords"})
            keyword = (key.get("content", None))
            keyword2 = keyword.replace(',','').split(" ")
            tags=keyword2 
            return tags
           
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
        
    if data['hostname'] in ['dailyjournalonline.com']:
        tags = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            key = soup.find("meta", attrs={"name":"news_keywords"})
            keyword = (key.get("content", None))
            tags.append(keyword) 
            return tags
           
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
        
    if data['hostname'] in ['www.audacy.com']:
        tags = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            key = soup.find_all("meta", attrs={"name":"branch:deeplink:editorial_tags"})
            for x in key:
                keyword = (x.get("content", None))
                tags.append(keyword) 
            return tags
        
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
        
        except IndexError:
            return ''
    
    if data['hostname'] in ['www.kprs.com']:
        tags = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            key = soup.find("a", attrs={"rel":"category tag"}).get_text()
            tags.append(key) 
            return tags[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
        
        except IndexError:
            return ''
        
    if data['hostname'] in ['www.kcur.org', 'news.stlpublicradio.org']:
        tags = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            key = soup.find_all("meta", attrs={"property":"article:tag"})
            for x in key:
                keyword = (x.get("content", None))
                tags.append(keyword) 
            return tags
        
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
        
        except IndexError:
            return ''
        
    if data['hostname'] in ['www.krcu.org']:
        tags = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            key = soup.find_all("meta", attrs={"property":"article:tag"})
            for x in key:
                keyword = (x.get("content", None))
                tags.append(keyword) 
            return tags[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
        
        except IndexError:
            return ''

    if data['hostname'] in ['www.newspressnow.com']:
        tags = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            key = soup.find("div", attrs={"class":"asset-tags hidden-print"})
            for x in key.find_all('a'):
                keyword = (x.get_text())
                tags.append(keyword) 
            return tags
        
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
        
        except IndexError:
            return ''
    
    elif data['hostname'] == 'www.kansascity.com' or 'www.kmbc.com':
        tags = data['meta_data']['keywords']
        return tags
    
    else: 
        pass

def given_location(item,data):
    if data['hostname'] == 'fox2now.com':
        loc = []
        soup = BeautifulSoup(article['text'],'html.parser')
        key = soup.find(attrs={"name":"primaryCategory"})
        # keyword = (key.get("content", None))
        loc.append(key)
        return loc

    
def metadata(item,data):
    # metadata = data['meta_data']['keywords']
    
    return metadata
    

def given_date(item,data):
    
    if data['hostname'] in ['www.redlatinastl.com']:
        date = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            txt = soup.find("li", attrs={"class":"text-muted"}).get_text().strip()
            txt_2 = datetime.strptime(txt, '%Y-%m-%d')
            date.append(str(txt_2))
            return date[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
    
    if data['hostname'] in ['cedarrepublican.com', 'ccheadliner.com']:
        date = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            txt = soup.find("span", attrs={"class":"monthday"}).get_text().strip()
            txt_2 = datetime.strptime(txt, '%B %d, %Y')
            date.append(str(txt_2))
            return date[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
    
    if data['hostname'] in ['www.bethanyclipper.com']:
        date = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            txt = soup.find("div", attrs={"class":"col-12 col-sm contact-section"}).get_text().replace('\n','')
            txt_2 = datetime.strptime(txt, '%A, %B %d, %Y')
            date.append(str(txt_2))
            return date[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return ''

    if data['hostname'] in ['www.mycaldwellcounty.com']:
        date = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            txt = soup.find("span", attrs={"property":"dc:date dc:created"})
            txt_cleaned = pd.to_datetime(txt["content"]).tz_convert('utc').tz_localize(None) 
            date.append(str(txt_cleaned))
            return date[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
        
    if data['hostname'] in ['moberlymonitor.com', 'gasconadecountyrepublican.com', 'www.kirksvilledailyexpress.com']:
        date = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("meta", {"name":"twt-published-at"})
            txt_date = txt.get("content", None)[:-6]
            #Tue, 11 Oct 2022 12:00:00
            txt_2 = datetime.strptime(txt_date, '%a, %d %b %Y %H:%M:%S')
            date.append(str(txt_2))
            return date[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
        
        except ValueError:
            return ''
    
    if data['hostname'] in ['www.kxcv.org']:
        date = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("p", {"class":"date"})
            txt_date = txt.find('b').get_text().replace('Sept.','Sep').replace('Jan.','Jan').replace('Feb.','Feb').replace('March','Mar').replace('April','Apr').replace('June','Jun').replace('July','Jul').replace('Oct.','Oct').replace('Nov.','Nov').replace('Dec.', 'Dec')
            txt_2 = datetime.strptime(txt_date, '%b %d, %Y')
            date.append(str(txt_2))
            return date[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
        
        except ValueError:
            return ''
        
    if data['hostname'] in ['www.theijnews.com']:
        date = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            txt = soup.find("div", {"class":"published-date"}).get_text().strip().replace(',','')
            #September 29, 2022
            txt_2 = datetime.strptime(txt, '%B %d %Y')
            date.append(str(txt_2))
            return date[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
        
        except ValueError:
            return ''
        
    if data['hostname'] in ['lewispnj.com','maconhomepress.com']:
        date = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            txt = soup.find("div", {"class":"published-date"}).get_text().strip()
            #September 29, 2022
            txt_2 = datetime.strptime(txt, '%B %d, %Y')
            date.append(str(txt_2))
            return date[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
        
        except ValueError:
            return ''
        
    if data['hostname'] == 'www.firesideguard.com':
        date = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            txt = soup.find("div", {"class":"pss-dateline"}).get_text().replace('Sunday,','').replace('Monday,','').replace('Tuesday,','').replace('Wednesday,','').replace('Thursday,','').replace('Friday,','').replace('Saturday,','').strip().replace('th','').replace('nd','').replace('rd','').replace('st','')
            #October 9, 2022
            txt_2 = datetime.strptime(txt, '%B %d, %Y')
            date.append(str(txt_2))
            return date[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
        
        except ValueError:
            return ''
    
    if data['hostname'] == 'www.legends1063.fm':
        date = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            txt = soup.find("p", {"class":"news-timestamp"}).get_text().strip().replace('th','').replace('nd','').replace('rd','').replace('st','').replace('am','AM').replace('pm','PM')
            #Thursday, September 22, 2022  6:21
            txt_2 = datetime.strptime(txt, '%A, %B %d, %Y  %I:%M%p')
            date.append(str(txt_2))
            return date[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
        
        except ValueError:
            return ''
        
    if data['hostname'] == 'www.4bcaonline.com':
        date = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            key = soup.find("meta", attrs={"property":"og:description"})
            date_text = (key.get("content", None))
            txt_2 = datetime.strptime(date_text, '%B %d, %Y')
            date.append(str(txt_2))
            return date[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
        
        except ValueError:
            return ''

    if data['hostname'] == 'unewsonline.com':
        date = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            txt = soup.find("span", {"class":"time-wrapper"}).get_text()
            txt_2 = datetime.strptime(txt, '%B %d, %Y')
            date.append(str(txt_2))
            return date[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
        
        except ValueError:
            return ''
    
    if data['hostname'] == 'www.theccreporter.com':
        date = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("p", string=re.compile(r', \d\d\d\d'))
            txt_cleaned = txt.get_text().replace('Published ','').replace('\n','').strip('\r')
            txt_2 = datetime.strptime(txt_cleaned, '%B %d, %Y')
            date.append(str(txt_2))
            return date[0]

        except AttributeError:
            return ''
        
        except TypeError:
            return ''   
    
    if data['hostname'] == 'themaneater.com':
        date = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("p", attrs={"class":"post-date"}).get_text()
            txt_cleaned = txt.replace('th','').replace('nd','').replace('rd','').replace('st','')
            txt_2 = datetime.strptime(txt_cleaned, '%B %d, %Y')
            date.append(str(txt_2))
            return date[0]

        except AttributeError:
            return ''
        
        except TypeError:
            return ''
    
    if data['hostname'] == 'www.riverfronttimes.com':
        date = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            txt = soup.find("meta", attrs={"name":"date"})
            date.append(txt["content"])
            return date[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
    
    if data['hostname'] == 'www.archstl.org':
        date = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            txt = soup.find("div", {"class":"col-12 col-lg-3 ml-auto publish-date-meta text-right"}).get_text()
            txt_cleaned = txt.split('|', 1)[0].replace('Submitted','').replace('\r','').replace('\n','').strip()
            txt_2 = datetime.strptime(txt_cleaned, '%B %d, %Y')
            date.append(str(txt_2))
            return date[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
        
        except ValueError:
            return ''
    
    if data['hostname'] == 'callnewspapers.com':
        date = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            txt = soup.find("span", {"class":"time-wrapper"}).get_text().strip('\n')
            txt_cleaned = datetime.strptime(txt, '%B %d, %Y')
            date.append(str(txt_cleaned))
            return date[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
        
        except ValueError:
            return ''
        
    if data['hostname'] in ['www.bentoncountyenterprise.com']:
        date = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            txt = soup.find("span", attrs={"property":"dc:date dc:created"})
            txt_cleaned = pd.to_datetime(txt["content"]).tz_convert('utc').tz_localize(None) 
            date.append(str(txt_cleaned))
            return date[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return ''

    if data['hostname'] in ['www.plattecountycitizen.com', 'the-standard.org']:
        date = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            txt = soup.find("meta", attrs={"itemprop":"datePublished"})
            txt_cleaned = pd.to_datetime(txt["content"]).tz_convert('utc').tz_localize(None) 
            date.append(str(txt_cleaned))
            return date[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return ''

    if data['hostname'] in ['www.timesnewspapers.com','www.bransontrilakesnews.com', 'stlamerican.com',
                           'www.stltoday.com', 'www.joplinglobe.com', 'www.maryvilleforum.com', 'websterkirkwoodtimes.com',
                           'www.newspressnow.com', 'dailyjournalonline.com', 'www.hermannadvertisercourier.com',
                           'www.lakegazette.net', 'emissourian.com']:
        date = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            txt = soup.find("meta", attrs={"itemprop":"dateCreated"})
            txt_cleaned = pd.to_datetime(txt["content"]).tz_convert('utc').tz_localize(None) 
            date.append(str(txt_cleaned))
            return date[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
     
    if data['hostname'] == 'sgfneighborhoodnews.com':
        date = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            txt = soup.find("span", {"class":"dtreviewed"}).get_text().strip('\n')
            txt_cleaned = datetime.strptime(txt, '%B %d, %Y')
            date.append(str(txt_cleaned))
            return date[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
        
        except ValueError:
            return ''
    
    if data['hostname'] == 'thesalemnewsonline.com':
        date = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            txt = soup.find("meta", attrs={"itemprop":"dateCreated"})
            txt_cleaned = pd.to_datetime(txt["content"]).tz_convert('utc').tz_localize(None) 
            date.append(str(txt_cleaned))
            return date[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
   
    if data['hostname'] == 'www.columbiamissourian.com':
        date = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            txt = soup.find("meta", attrs={"name":"parsely-pub-date"})
            txt_cleaned = pd.to_datetime(txt["content"]).tz_convert('utc').tz_localize(None) 
            date.append(str(txt_cleaned))
            return date[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
    
    if data['hostname'] == 'missouri.statenews.net':
        date = []
        soup = BeautifulSoup(article['text'],'html.parser') 
        key = soup.find(attrs={'class':'title_text'})
        temp = key.p.get_text(strip=True, separator='\n').splitlines()
        date.append(temp[1])
        return date
    
    elif data['hostname'] == 'www.kansascity.com':
        date = []
        soup = BeautifulSoup(article['text'],'html.parser')
        div = soup.find('div', attrs={'class':'publish-date'})
        srce = div.contents[0]
        date.append(srce)
        return date
    
    if data['hostname'] == 'www.gasconadecountyrepublican.com':
        date = data['meta_data']['twt-published-at']
        return date
    
    if data['hostname'] == 'www.hannibal.net':
        date = []
        soup = BeautifulSoup(article['text'],'html.parser')
        key  = soup.find('meta', attrs={'name': 'tncms-access-version'})
        content = key['content']
        date.append(content)
        return date
    
    else:
        
        date = data['date']
        return date
    

def source(item,data):
    if data['hostname'] == 'missouri.statenews.net': 
        src = []
        soup = BeautifulSoup(article['text'],'html.parser') 
        key = soup.find(attrs={'class':'title_text'})
        temp = key.p.get_text(strip=True, separator='\n').splitlines()
        src.append(temp[0])
        return src
    
    if data['hostname'] == 'abc17news.com':
        src = []
        soup = BeautifulSoup(article['text'],'html.parser')
        div = soup.find('div', attrs={'class':'meta__category'})
        srce = div.find('a').contents[0]
        src.append(srce)
        return src
    
    else: 
        
        pass


def author(item,data):
    
    if data['hostname'] in ['bocojo.com']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"class" : "c5-white-box layout-2 c5-margin clearfix"})
            x = results.find_all('p')
            for y in x:
                if y.get_text().startswith('By:'):
                    auth.append(y.get_text())
            return ' '.join(auth).replace('By:','').strip()
        
        except AttributeError:
            return ''
        except TypeError:
            return ''

    if data['hostname'] in ['ccheadliner.com']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("meta", attrs={"name":"twt-author-name"})
            txt2 = (txt.get("content", None)).replace('By','').strip().split("\n",1)[0].lower()
            auth.append(txt2)
            return auth[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return '' 
    
    if data['hostname'] in ['moberlymonitor.com', 'cedarrepublican.com', 'gasconadecountyrepublican.com', 
                            'www.kirksvilledailyexpress.com']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("meta", attrs={"name":"twt-author-name"})
            txt2 = (txt.get("content", None)).split('\n',1)[0].replace('By','').strip()
            auth.append(txt2)
            return auth[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return '' 
    
    if data['hostname'] in ['maconhomepress.com']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("div", attrs={"class":"author"})
            x = txt.get_text().replace('by','').strip()
            auth.append(x)
            return auth[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return '' 

    if data['hostname'] in ['www.mycaldwellcounty.com']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("div", attrs={"property":"content:encoded"})
            x = txt.find('a').get_text()
            auth.append(x)
            return auth[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return '' 

    if data['hostname'] in ['www.excelsiorspringsstandard.com']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("div", attrs={"class":"user-name"})
            x = txt.find('a').get_text()
            auth.append(x)
            return auth[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return '' 

    if data['hostname'] in ['www.dixonpilot.com', 'www.edinasentinel.com']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("span", attrs={"itemprop":"name"}).get_text()
            auth.append(txt)
            return auth[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return '' 
    
    if data['hostname'] in ['www.kxcv.org']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("i").get_text().encode('ascii', 'ignore').decode('ascii')
            auth.append(txt)
            return auth[0]

        except AttributeError:
            return ''
        
        except TypeError:
            return '' 

    if data['hostname'] in ['www.kprs.com']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("span", attrs={"class":"single-meta"}).get_text()
            txt2 = txt.replace('|','').strip()
            auth.append(txt2)
            return auth[0]

        except AttributeError:
            return ''
        
        except TypeError:
            return '' 

    if data['hostname'] in ['www.kbia.org']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("meta", attrs={"name":"gtm-dataLayer"})
            txt2 = txt["content"].strip()
            txt_dict = json.loads(txt2)
            auth.append(txt_dict['gtmAuthor'].strip())
            return auth[0]

        except AttributeError:
            return ''
        
        except TypeError:
            return ''  
    
    if data['hostname'] in ['www.kcur.org', 'news.stlpublicradio.org', 'www.ksmu.org', 'www.krcu.org']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("meta", attrs={"name":"gtm-dataLayer"})
            txt2 = txt["content"].strip()
            txt_dict = json.loads(txt2)
            auth.append(txt_dict['gtmAuthor'].strip())
            return auth[0]

        except AttributeError:
            return ''
        
        except TypeError:
            return ''   
    
    
    if data['hostname'] == 'www.legends1063.fm':
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("p", attrs={"class":"news-author"})
            auth.append(txt.get_text().replace('By','').replace('\n','').strip().split('  ', 1)[0])
            return auth[0]

        except AttributeError:
            return ''
        
        except TypeError:
            return ''
    
    if data['hostname'] == 'themaneater.com':
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("p", attrs={"class":"post-author"})
            auth.append(txt.get_text().strip('By '))
            return auth[0]

        except AttributeError:
            return ''
        
        except TypeError:
            return ''
    
    if data['hostname'] in ['www.southeastarrow.com', 'www.nevadadailymail.com']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("div", attrs={"class":"byline"})
            auth.append(txt.get_text())
            return auth[0]

        except AttributeError:
            return ''
        
        except TypeError:
            return ''
    
    if data['hostname'] == 'www.archstl.org':
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        #selector = 'div.col-8 > h3'
        try:
            txt = soup.find("div", attrs={"class":"author-meta d-flex flex-wrap justify-content-center flex-lg-column justify-content-lg-start"})
            auth.append(txt.get_text().replace('\n','').replace('\r','').strip())
            return auth[0]

        except AttributeError:
            return ''
        
        except TypeError:
            return ''
    
    if data['hostname'] == 'www.news-leader.com':
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("meta", attrs={"property":"article:author"})
            auth.append(txt["content"])
            return auth[0]

        except AttributeError:
            return ''
        
        except TypeError:
            return ''
    
    if data['hostname'] == 'www.semissourian.com':
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find('div', {'class':['byline']}).get_text()
            auth.append(txt["content"])
            return auth[0]

        except AttributeError:
            return ''
        
        except TypeError:
            return ''
        
        try:
            txt = soup.find('a', {'class':['columnistbyline']}).get_text()
            auth.append(txt["content"])
            return auth[0]

        except AttributeError:
            return ''
        
        except TypeError:
            return ''
    
    if data['hostname'] in ['www.timesnewspapers.com','www.bransontrilakesnews.com']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("span", {"class":"reviewer"}).get_text().strip()
            auth.append(txt)
            return auth[0]

        except AttributeError:
            return ''
        
        except TypeError:
            return ''
    
    if data['hostname'] == 'sgfneighborhoodnews.com':
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("span", {"class":"reviewer"}).get_text().strip()
            auth.append(txt)
            return auth[0]

        except AttributeError:
            return ''
        
        except TypeError:
            return ''
    
    if data['hostname'] in ['thesalemnewsonline.com', 'stlamerican.com', 'www.pikecountynews.com', 
                            'newspressnow.com', 'www.newspressnow.com', 'callnewspapers.com', 'www.stltoday.com', 'www.riverfronttimes.com',
                           'www.joplinglobe.com', 'websterjournal.com', 'plattecountylandmark.com', 
                           'www.maryvilleforum.com', 'the-standard.org', 'unewsonline.com', 'websterkirkwoodtimes.com', 
                           'www.thecash-book.com', 'dailyjournalonline.com', 'www.hermannadvertisercourier.com',
                           'www.lakegazette.net', 'emissourian.com', 'molawyersmedia.com']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("meta", attrs={"name":"author"})
            auth.append(txt["content"].strip())
            return auth[0]

        except AttributeError:
            return ''
        
        except TypeError:
            return ''
    
    if data['hostname'] == 'lstribune.net':
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt =  soup.find("strong").get_text()
            if txt.startswith('By'):
                auth.append(txt)
                return auth[0]

        except AttributeError:
            return ''
        
        except TypeError:
            return ''
    
    
    if data['hostname'] == 'www.myleaderpaper.com':
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("span", {"class":"tnt-byline"}).get_text().strip()
            auth.append(txt)
            return auth[0]

        except AttributeError:
            return ''
        
        except TypeError:
            return ''
    
    if data['hostname'] in ['www.newstribune.com', 'www.fultonsun.com', 'www.plattecountycitizen.com']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("meta", attrs={"itemprop":"author"})
            auth.append(txt["content"])
            return auth[0]    

        except AttributeError:
            return ''
        
        except TypeError:
            return ''
    
    if data['hostname'] == 'www.greenecountycommonwealth.com':
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find('div', class_='field-item even').get_text()
            if len(txt) <50:
                auth.append(txt)
                return auth[0]
            else: 
                return ''
            

        except AttributeError:
            return ''
        
        except TypeError:
            return ''
    
    if data['hostname'] == 'www.dddnews.com':
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find('div', {'class':'byline'}).get_text()
            auth.append(txt)
            return auth[0]

        except AttributeError:
            return ''
        
        except TypeError:
            return ''
    
    if data['hostname'] == 'dailyjournalonline.com':
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("span", {"itemprop":"author"}).get_text().strip()
            auth.append(txt)
            return auth[0]

        except AttributeError:
            return ''
        
        except TypeError:
            return ''
    
    
    if data['hostname'] == 'www.darnews.com':
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find('div', {'class':'byline'}).get_text()
            auth.append(txt)
            return auth[0]

        except AttributeError:
            return ''
        
        except TypeError:
            return ''

    if data['hostname'] == 'www.columbiamissourian.com':
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("meta", attrs={"name":"author"})
            auth.append(txt["content"])
            return auth[0]

        except AttributeError:
            return ''
        
        except TypeError:
            return ''

    if data['hostname'] == 'www.audacy.com':
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("meta", property="article:author:name")
            auth.append(txt["content"])
            return auth[0]

        except AttributeError:
            return ''
        
        except TypeError:
            return ''
    
    if data['hostname'] in ['www.columbiatribune.com', 'newstalkkzrg.com']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("meta", property="article:author")
            auth.append(txt["content"])
            return auth[0]

        except AttributeError:
            return ''
        
        except TypeError:
            return ''
       
    
    if data['hostname'] == 'mycameronnews.com':
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find('div', {'class':'field field-name-field-dateline field-type-text field-label-inline clearfix'}).get_text().strip('By:\xa0')
            auth.append(txt)
            return auth[0]

        except AttributeError:
            return ''
    
    if data['hostname'] == 'www.cassville-democrat.com':
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find('div', {'class':['byline']}).get_text()
            auth.append(txt)
            return auth[0]

        except AttributeError:
            return ''
    
    if data['hostname'] == 'carthagenewsonline.com':
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("div", {"class": "td-post-author-name"})
            anchor = txt.find('a').get_text()
            auth.append(anchor)
            return auth[0]

        except AttributeError:
            return ''

    if data['hostname'] == 'www.californiademocrat.com':
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find('a', attrs={'class':'bi-line-link'}).get_text()
            auth.append(txt)
            return auth[0]

        except AttributeError:
            return ''
    
    if data['hostname'] == 'auroraadvertiser.net':
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        spans = soup.find('span', attrs={'class':'byline'})
        auth.append(spans.text)

    
    if data['hostname'] == 'www.bransontrilakesnews.com':
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("span", { "itemprop" : "author" }).get_text()
            auth.append(txt)
            return auth[0]

        except AttributeError:
            return auth
    
    if data['hostname'] == 'www.memphisdemocrat.com':
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        txt_byline = soup.find(attrs={'class':'p2'})
        txt = soup.find("span", itemprop="name").text
        try:
            cleaned = txt_byline.get_text()
            if cleaned.startswith('By'): 
                auth.append(cleaned) 
                return auth[0]
           
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
    
    
    if data['hostname'] == 'www.ksdk.com':
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        txt = soup.select_one('span:-soup-contains("Author:")').find_next_sibling(text=True)
        aut = txt.strip()
        auth.append(aut)
        return auth
    
    if data['hostname'] == 'dosmundos.com':
        auth = []
        try:
            soup = BeautifulSoup(article['text'], "html.parser")
            results=soup.find("div", {"class" : "elementor-element elementor-element-18433643 elementor-widget elementor-widget-theme-post-content"}).get_text()
            x = re.findall(r'(By\s\w+\s[A-Z].*?(?=[A-Z]))', results)
            auth.append(x)
            y = auth[0]
            return ' '.join(y).split('\n',1)[0].split('and',1)[0].replace('By','').strip()
        
        except AttributeError:
            return ''
        except TypeError:
            return ''
    
    if data['hostname'] in ['houstonherald.com', 'stljewishlight.org', 'www.marshallnews.com', 
                           'www.monett-times.com', 'mbcpathway.com', 'www.thepitchkc.com', 
                           'www.mymoinfo.com']:
        auth= []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            txt = soup.find("meta", attrs={"name":"twitter:data1"})
            auth.append(txt["content"].strip())
            return auth[0]
           
        except AttributeError:
            return ''
        
        except TypeError:
            return ''

    
    if data['hostname'] == 'www.gasconadecountyrepublican.com':
        auth = []
        aut = data['meta_data']['twt-author-name']
        word_list = aut
        auth = word_list[3:]
        return auth
    
    if data['hostname'] == 'www.hannibal.net':
        auth = data['meta_data']['author']
        return auth

    else:
        
        author = data['author']
        return author
    
def news(item,data): 
    
    if data['hostname'] in ['molawyersmedia.com']:
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"class" : "entry entry-content"})
            y = results.find_all('p')
            for z in y:
                nws.append(z.get_text())
            return ' '.join(nws).encode('ascii', 'ignore').decode('ascii').replace('\n','')    
        
        except AttributeError:
            return ''
        except TypeError:
            return ''
    
    if data['hostname'] in ['dosmundos.com']:
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"class" : "elementor-element elementor-element-18433643 elementor-widget elementor-widget-theme-post-content"})
            y = results.find('strong').get_text()
            z = re.sub(r'(By\s\w+\s[A-Z].*?(?=[A-Z]))', '', y)
            return ''.join(z)    
        
        except AttributeError:
            return ''
        except TypeError:
            return ''
    
    if data['hostname'] in ['www.redlatinastl.com']:
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("section", {"class" : "blog_area single-post-area section-padding"})
            y = results.find_all('p')
            for z in y:
                nws.append(z.get_text())
            return ' '.join(nws).encode('ascii', 'ignore').decode('ascii').replace('\n','')    
        
        except AttributeError:
            return ''
        except TypeError:
            return ''
    
    if data['hostname'] in ['cedarrepublican.com', 'ccheadliner.com']:
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"class" : "body main-body clearfix"})
            x = results.find_all('p', {"class":"p1"})
            for y in x:
                nws.append(y.get_text())
            return ' '.join(nws).encode('ascii', 'ignore').decode('ascii').replace('\n','')
            
        
        except AttributeError:
            return ''
        except TypeError:
            return ''
    
    if data['hostname'] in ['bocojo.com']:
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"class" : "c5-white-box layout-2 c5-margin clearfix"})
            x = results.find_all('p')
            for y in x:
                if y.get_text().startswith('By:'):
                    continue
                nws.append(y.get_text())
            return ' '.join(nws).encode('ascii', 'ignore').decode('ascii').replace('\n','').split('See more in', 1)[0].strip()
        
        except AttributeError:
            return ''
        except TypeError:
            return ''
    
    if data['hostname'] in ['www.bethanyclipper.com']:
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"class" : "entry-content"})
            x = results.find_all('p')
            for y in x:
                nws.append(y.get_text())
            return ' '.join(nws).encode('ascii', 'ignore').decode('ascii').replace('\n','').split('How useful', 1)[0].strip()
        
        except AttributeError:
            return ''
    
    if data['hostname'] in ['moberlymonitor.com', 'gasconadecountyrepublican.com', 'www.kirksvilledailyexpress.com']:
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"class" : "body main-body clearfix"})
            x = results.find_all('p')
            for y in x:
                nws.append(y.get_text())
            return ' '.join(nws).encode('ascii', 'ignore').decode('ascii').replace('\n','')
        
        except AttributeError:
            return ''
    
    if data['hostname'] in ['www.dixonpilot.com']:
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"itemprop" : "text"})
            x = results.find_all('p')
            for y in x:
                nws.append(y.get_text())
            return ' '.join(nws).encode('ascii', 'ignore').decode('ascii').replace('\n','')
        
        except AttributeError:
            return ''
    
    if data['hostname'] in ['www.edinasentinel.com']:
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"itemprop" : "text"})
            x = results.find_all('p')
            for y in x:
                nws.append(y.get_text())
            return ' '.join(nws).encode('ascii', 'ignore').decode('ascii').replace('\n','').replace('If youre a current subscriber, log in below. If you would like to subscribe, please click the subscribe tab above.  Username    Password       Login   Lost your password?  Please enter your email and we will send your username and password to you.  Email    Submit   Back ','').strip()
        
        except AttributeError:
            return ''
    
    if data['hostname'] in ['dailyjournalonline.com']:
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"itemprop" : "articleBody"})
            x = results.find_all('p')
            for y in x:
                nws.append(y.get_text())
            return ' '.join(nws).encode('ascii', 'ignore').decode('ascii').split('  Get', 1)[0]
        
        except AttributeError:
            return ''
    
    if data['hostname'] == 'www.ccscrnews.com':
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find_all("div", {"class" : "paragraph"})
            for x in results:
                nws.append(x.get_text())
            return ' '.join(nws).encode('ascii', 'ignore').decode('ascii').split("All content copyright", 1)[0].replace('Braden Clines','').replace('Kadee Brosseau DeCourley','').strip()
            
        except AttributeError:
            return '' 
    
    if data['hostname'] == 'www.4bcaonline.com':
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find_all("div", {"class" : "paragraph"})
            for x in results:
                nws.append(x.get_text())
            return ' '.join(nws).encode('ascii', 'ignore').decode('ascii').split(" Barry County AdvertiserOffice:", 1)[0].replace('Adriana Keeton','')
            
        except AttributeError:
            return '' 
        
    if data['hostname'] in ['lewispnj.com', 'maconhomepress.com','www.theijnews.com']:
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"class" : "article-main"})
            for p in results.find_all('p'):
                nws.append(p.get_text())
            return ' '.join(nws).encode('ascii', 'ignore').decode('ascii')
            
        except AttributeError:
            return ''
    
    if data['hostname'] == 'www.ktts.com':
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"class" : "jeg-content-inner"})
            for p in results.find_all('p'):
                nws.append(p.get_text())
            return ' '.join(nws).encode('ascii', 'ignore').decode('ascii')
            
        except AttributeError:
            return '' 
    
    if data['hostname'] == 'www.nevadadailymail.com':
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"class" : "content"})
            for p in results.find_all('p'):
                nws.append(p.get_text())
            return ' '.join(nws).encode('ascii', 'ignore').decode('ascii')
            
        except AttributeError:
            return '' 
    
    if data['hostname'] == 'themaneater.com':
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"class" : "wp-container-2 wp-block-column"})
            for p in results.find_all('p'):
                nws.append(p.get_text())
            return ' '.join(nws)
            
        except AttributeError:
            return '' 
    
    
    if data['hostname'] == 'plattecountylandmark.com':
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"class" : "content-inner"})
            for p in results.find_all('p'):
                nws.append(p.get_text())
            return ' '.join(nws)
            
        except AttributeError:
            return '' 
    
    
    if data['hostname'] == 'www.southeastarrow.com':
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"class" : "text"})
            for p in results.find_all('p'):
                nws.append(p.get_text())
#.encode('ascii', 'ignore').decode('ascii') removes the \x91, etc. characters
            return ' '.join(nws).encode('ascii', 'ignore').decode('ascii')
            
        except AttributeError:
            return '' 
    
    if data['hostname'] == 'www.semissourian.com':
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find_all('p')
            nws.append(results.get_text())
            return nws[0]
            
        except AttributeError:
            return '' 

    if data['hostname'] == 'www.clintoncountyleader.com':
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find_all("div", {"class" : "paragraph"})
            for res in results:
                nws.append(res.text)
                return nws[0]
            
        except AttributeError:
            return '' 

    if data['hostname'] == 'stlamerican.com':
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        sep = 'TNCMS.AdManager.render'
        try:
            results=soup.find_all("div", {"itemprop" : "articleBody"})
            for res in results:
                nws.append(res.text.split(sep, 1)[0])
                return nws[0]
        
        except AttributeError:
            return '' 

    if data['hostname'] == 'www.joplinglobe.com':
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        sep = ';\n\n'
        sep2 = '\n\n\n'
        try:
            results=soup.find_all("div", {"itemprop" : "articleBody"})
            for res in results:
                nws.append(res.text.split(sep, 1)[1].split(sep, 1)[1].split(sep2, 1)[0])
                return nws[0]
        
        except AttributeError:
            return '' 
    
    if data['hostname'] in ['www.hermannadvertisercourier.com','www.lakegazette.net']:
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"itemprop" : "articleBody"})
            for p in results.find_all('p'):
                nws.append(p.get_text())
            return ' '.join(nws).replace('\n','')
        
        except AttributeError:
            return ''
    

    if data['hostname'] in ['www.timesnewspapers.com','www.bransontrilakesnews.com', 'www.columbiamissourian.com', 
                           'dailyjournalonline.com', 'www.maryvilleforum.com', 'websterkirkwoodtimes.com', 
                           'emissourian.com']:
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find_all("div", {"itemprop" : "articleBody"})
            for res in results:
                nws.append(res.text.replace('\n',''))
                return nws[0]
        
        except AttributeError:
            return ''

    if data['hostname'] in ['the-standard.org']:
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find_all("div", {"itemprop" : "articleBody"})
            for res in results:
                nws.append(res.text)
                return nws[0].encode('ascii', 'ignore').decode('ascii')
        
        except AttributeError:
            return ''          
    
    if data['hostname'] == 'standard-democrat.com':
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results = soup.find('div',attrs={"class":"text"})
            nws.append(results.get_text())
            return nws[0]
        
        except AttributeError:
            return ''
        
        try:
            results_paywall = soup.find('div',attrs={"class":"pwblock"})
            nws.append(results.get_text())
            return nws[0]
        
        except AttributeError:
            return ''

    if data['hostname'] == 'thesalemnewsonline.com':
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find_all("div", {"itemprop" : "articleBody"})
            for res in results:
                nws.append(res.text)
                return nws[0]
        
        except AttributeErrdeor:
            return ''
    
    
    if data['hostname'] == 'www.pikecountynews.com':
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find_all("div", {"itemprop" : "articleBody"})
            for res in results:
                nws.append(res.text)
                return nws[0]
        
        except AttributeError:
            return ''
    
    if data['hostname'] in ['newspressnow.com']:
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"itemprop" : "articleBody"})
            for p in results.find_all('p'):
                nws.append(p.get_text())
            return ' '.join(nws)
        
        except AttributeError:
            return ''
        
    if data['hostname'] == 'www.clintoncountyleader.com':
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find('div', {'class':'blog-content'}).get_text()
            nws.append(txt)
            return nws[0]
        
        except AttributeError:
            return ''
        
    else:
        news = data['news']
        return news


def metadescription(item,data):
    if data['hostname'] == 'dosmundos.com':
        desc = []
        soup = BeautifulSoup(article['text'], "html.parser")
        descrip = soup.find("meta", property="og:description")
        des = descrip["content"]
        desc.append(des)
        return desc
    
def title(item,data):
    if data['hostname'] == 'www.firesideguard.com':
        title = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find('h1').get_text()
            title.append(txt)
            return title[0]
        
        except AttributeError:
            return ''
    

def source_1(item,data):
    if data['hostname'] == 'abc17news.com' and 'fox2now.com':
        src = []
        soup = BeautifulSoup(article['text'],'html.parser')
        div = soup.find('div', attrs={'class':'meta__user vcard author'})
        srce = div.find('a').contents[0]
        src.append(srce)
        return src

news
def process_article(item):
    article = Article(url=item["url"])
    article.download(input_html=item["text"])

    article.parse()
    article.nlp()
    url = urlparse(item["url"])
        # print(hashlib.sha256(item['url'].encode()).hexdigest)
    data = {
            "id": hashlib.sha256(item["url"].encode()).hexdigest(),
            "title": str(article.title),
            "url": str(item["url"]),
            "news": str(article.text),
            "image": str(article.top_image),
            "date": str(article.publish_date),
            "author": article.authors,                                   # author derived from article.authors
            "meta_data": article.meta_data,
            "hostname": url.hostname,
            "crawl_datetime": str(datetime.now()),
            "stats": {},
            "tags" : {},                                                   # tags given by the outlet/reporter
            "inferred_tags_set1" : str(article.keywords),                 # tags derived from article.keywords
            "inferred_tags_set2" : {},                                    # tags inferred by our custom method
        
        }
    
    if len(data["news"]) > 10 and len(data["title"]) > 0:
        data['stats']['num_char'] = len(data['news'])
        
        date = given_date(item,data)
        data['date'] = date
        # if date =='None':
        #     data['date'] = str(article.publish_date)
        
        tags = given_tags(item,data)
        data['tags'] = tags
 
        #src = source(item,data)
        #data['source'] = src
        
        #src = source_1(item,data)
        #data['source_1'] = src
        
        auth = author(item,data)
        data['author'] = auth
        
        nws = news(item,data)
        data['news'] = nws
    
        
    if len(data["title"]) == 0:
        headline = title(item,data)
        data['title'] = headline
    
    if data['hostname'] in ['www.firesideguard.com']:
        date = given_date(item,data)
        data['date'] = date
    
    if data['hostname'] in ['lewispnj.com', 'maconhomepress.com', 'www.theijnews.com']:
        nws = news(item,data)
        data['news'] = nws
        
    if data['hostname'] in ['www.redlatinastl.com']:
        date = given_date(item,data)
        data['date'] = date
        
    if data['hostname'] in ['maconhomepress.com', 'www.theijnews.com']:
        auth = author(item,data)
        data['author'] = auth
        d
        date = given_date(item,data)
        data['date'] = date
    
    if data['hostname'] == 'www.4bcaonline.com':
        nws = news(item,data)
        data['news'] = nws
        
        date = given_date(item,data)
        data['date'] = date
        
        auth = author(item,data)
        data['author'] = auth
        
    if data['hostname'] == 'bocojo.com':
        nws = news(item,data)
        data['news'] = nws
        
    if data['hostname'] == 'www.redlatinastl.com':
        nws = news(item,data)
        data['news'] = nws
        
    if data['hostname'] == 'molawyersmedia.com':
        nws = news(item,data)
        data['news'] = nws
        
    return data
    

In [ ]:
curated_articles = []
for article in raw_articles: 
    curated_article = process_article(article)
    curated_articles.append(curated_article)

In [ ]:
curated_articles